In [15]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.integrate as integrate

def Prior(p):
    return np.piecewise( p, [p>= 0 and p <= 1, p<0 and p > 1], [lambda p: 1, lambda p:0])
Prior = np.vectorize(Prior)

def Likelihood(p,r,n):
    return p**r*(1-p)**(n-r)

def Posterior(p,r,n):
    return Likelihood(p,r,n)*Prior(p)

def Metropolis(x0, Posterior, NSteps=int(1e4), delta= 0.4):
    
    x = np.zeros((NSteps,1))
    
    # Prior
    x[0] = x0
    
    for i in tqdm(range(1,NSteps)):
        
        P0 = Posterior(x[i-1],r,n)
        
        xf = x[i-1] + delta*2*(np.random.rand()-0.5)
        
        P1 = Posterior(xf,r,n)
        
        alpha = np.minimum( 1, P1/P0 )
        g = np.random.rand()
        
        if alpha > g:
            x[i,0] = xf
        else:
            x[i,:] = x[i-1,:]
            
    return x[1000:,:]

p = np.linspace(0,1,100)
r = 7
n = 10
Pos = Posterior(p,r,n)
I,_ = integrate.quad(Posterior,p[0],p[-1],args=(r,n))
initparams = np.array([0.2])
MCMC = Metropolis(initparams,Posterior)
liminf = np.percentile(MCMC,16)
limsum = np.percentile(MCMC,84)
mean = np.percentile(MCMC,50)
confidence = 68
conclusion = "p̂ = {:.2f} ± {:.2f} a un nivel de confianza del {}%"
print(conclusion.format(mean, (limsum - mean), confidence))
print("Si la moneda no estuviera truncada, p̂ debería acercarse a 0.5. En este caso, obtuvimos un valor bastante alejado, p̂ = 0.68.\nPor tanto, se concluye que la moneda sí está truncada.")

100%|██████████| 9999/9999 [00:02<00:00, 4131.89it/s]

p̂ = 0.68 ± 0.12 a un nivel de confianza del 68%
Si la moneda no estuviera truncada, p̂ debería acercarse a 0.5. En este caso, obtuvimos un valor bastante alejado, p̂ = 0.68.
Por tanto, se concluye que la moneda sí está truncada.
